In [6]:
from square.client import Client
import pandas as pd

In [7]:
## Add this to the first block in your note book to display json
# another option: 
import uuid
from IPython.core.display import display, HTML

import json

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json_data
        self.uuid = str(uuid.uuid4())
        # This line is missed out in most of the versions of this script across the web, it is essential for this to work interleaved with print statements
        self._ipython_display_()
        
    def _ipython_display_(self):
        display(HTML('<div id="{}" style="height: auto; width:100%;"></div>'.format(self.uuid)))
        display(HTML("""<script>
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
          renderjson.set_show_to_level(1)
          document.getElementById('%s').appendChild(renderjson(%s))
        });</script>
        """ % (self.uuid, self.json_str)))

# Since this is copy-pasted wrongly(mostly) at a lot of places across the web, i'm putting the fixed, updated version here, mainly for self-reference


## To use this function, call this, this now works even when you have a print statement before or after the RenderJSON call
# RenderJSON(your_json)
# RenderJSON(json.loads(result.text))

In [8]:
# module
# sandbox connection to "Square SandBox no2"

from square.client import Client
sandbox_token = "EAAAECCddSeDNsweJXdjNVux8FTfVgMUXAroCh7VcOuoPlgjYb7MrkULH8DZrZmJ"

client = Client(
    access_token = sandbox_token,
    environment='sandbox',
)

In [31]:
# module: create order based on item_id (i.e., catalog_object_id)
# prerequisite: catalog_object_id is known
# note: by default the state is "OPEN", which would not shown in the dashboard order session 
# (only shown "Acitve", "Completed", "Upcoming")
idempotency_key = str(uuid.uuid4())
result = client.orders.create_order(
  body = {
    "order": {
      "location_id": "LCN0KMDE4Y5WR",
      "state": "OPEN"
      "line_items": [
        {
          "quantity": "1",
          "catalog_object_id": "IRC3NJWX5QBVIW7U736JPRCC"
        },
          {
          "quantity": "2",
          "catalog_object_id": "KYZZN57QA5JHS6CWZBMWMEE6"
        }
      ]
    },
    "idempotency_key": idempotency_key
  }
)

if result.is_success():
  RenderJSON(json.loads(result.text))
elif result.is_error():
  print(result.errors)

In [30]:
# module: search order (based on location_id)
result = client.orders.search_orders(
  body = {
    "location_ids": [
      "LCN0KMDE4Y5WR"
    ]
  }
)

if result.is_success():
  RenderJSON(json.loads(result.text))
elif result.is_error():
  print(result.errors)

In [36]:
# retrieve by order_id
# o2QhPTMErWXvoxvwCfvDNuM8Nh4F
# note: payment_id (appear as "tenders"."id": Ni0fotwm56cpIcV8kFMdqA3y8N6YY) is accessible if state: "COMPLETED"
result = client.orders.retrieve_order(
  order_id = "o2QhPTMErWXvoxvwCfvDNuM8Nh4F"
)

if result.is_success():
  RenderJSON(json.loads(result.text))
elif result.is_error():
  print(result.errors)

In [37]:
# module pay order in pyments.create_payment entry
# prerequiste: order_id is known (along with the total amount, 
# assocaited with the "order".get("total_money").get("amount") field)
# note: 
# - only works on order that the state is "OPEN"
# - the payment amount needs to match order amount
# - the Orders session in dashboard seems still not working after completed. 
#   my guess is Order Session only deal with order with tags like Delivery, pickup, Shipment.
# - The good news is in Transaction session in dashboard it appear a new transaction. 
#   and we can export orders and transactions in spread-sheet format (In sandbox and production)
idempotency_key = str(uuid.uuid4())

order_id = "o2QhPTMErWXvoxvwCfvDNuM8Nh4F"
result = client.payments.create_payment(
  body = {
    "source_id": "cnon:card-nonce-ok",
    "idempotency_key": idempotency_key,
    "amount_money": {
      "amount": 6000,
      "currency": "USD"
    },
    "order_id": order_id,
#     "reference_id": "123456"
  }
)

if result.is_success():
  RenderJSON(json.loads(result.text))
elif result.is_error():
  print(result.errors)

[{'category': 'INVALID_REQUEST_ERROR', 'code': 'BAD_REQUEST', 'detail': 'The order is already paid.'}]


In [95]:
###############
# module: wrapper order and pay in one go


def create_order_then_pay_wrapper(line_items_list, location_id, customer_id=None):
#     create_order
    result_create_order = client.orders.create_order(
      body = {
        "order": {
          "location_id": location_id,
            "customer_id": customer_id,
          "state": "OPEN",
          "line_items": line_items_list
        },
        "idempotency_key": str(uuid.uuid4())
      }
    )
# if create_order succeed then create_payment (using default source_id: cnon:card-nonce-ok)
    if result_create_order.is_success():    
        order_id = result_create_order.body.get("order").get("id")
        amount = result_create_order.body.get("order").get("total_money").get("amount")
        print("create_order SUCCEED")
    elif result_create_order.is_error():
        print(result_create_order.errors)
# #  create a automatically created note here, e.g., [2 x hat, 3 x shirt]
#     def line_items_trim(line_items):
#         return [[item.get("catalog_object_id"), 
#                            item.get("base_price_money").get("amount"),
#                            item.get("quantity"),
#                            item.get("name"),
#                            item.get("variation_name")] for item in line_items]
#     line_items_trim(result_create_order.body.get("order").get("line_items"))
#     note_payment = ",".join([f"{item_order[2]} x {item_order[4]}" for item_order in line_items_trim(result_tmp)])
# #     note_payment

    result_create_payment = client.payments.create_payment(
      body = {
        "source_id": "cnon:card-nonce-ok",
        "idempotency_key": str(uuid.uuid4()),
        "amount_money": {
          "amount": amount,
          "currency": "USD"
        },
        "order_id": order_id,
#           "note": note_payment
      }
    )

    if result_create_payment.is_success():
        print("create_payment SUCCEED")
    elif result_create_payment.is_error():
        print(result_create_payment.errors)
    return result_create_order, result_create_payment


# test here
line_items_list = [
        {
          "quantity": "10",
          "catalog_object_id": "IRC3NJWX5QBVIW7U736JPRCC"
        },
          {
          "quantity": "11",
          "catalog_object_id": "KYZZN57QA5JHS6CWZBMWMEE6"
        }
      ]
location_id = "LCN0KMDE4Y5WR"
customer_id = "RAWRMY0ZRWZVKC51JH2JQT2PKW"

result_create_order, result_create_payment = create_order_then_pay_wrapper(line_items_list, location_id, customer_id)


create_order SUCCEED
create_payment SUCCEED


In [93]:
result_tmp = result_create_order.body.get("order").get("line_items")
result_tmp
def line_items_trim(line_items):
    return [[item.get("catalog_object_id"), 
                       item.get("base_price_money").get("amount"),
                       item.get("quantity"),
                       item.get("name"),
                       item.get("variation_name")] for item in line_items]
line_items_trim(result_create_order.body.get("order").get("line_items"))
note_payment = [f"{item_order[2]} x {item_order[4]}" for item_order in line_items_trim(result_tmp)]
note_payment

['3 x Blu hat', '6 x Small red shirt']

In [85]:
RenderJSON(result_create_order.body)

RenderJSON((client.orders.retrieve_order("W5SDFdTFSEPQuPATz4mLwkHEta4F")).body)

In [39]:
# how do we know which payments is associated with the order?
# module retrive payment
# note the order_id can be retrived from get_payment
result = client.payments.get_payment(payment_id ="Ni0fotwm56cpIcV8kFMdqA3y8N6YY")

if result.is_success():
  RenderJSON(json.loads(result.text))
elif result.is_error():
  print(result.errors)

In [40]:
# transaction (deprecated)
result = client.transactions.retrieve_transaction(
  location_id = "LCN0KMDE4Y5WR",
  transaction_id = "o2QhPTMErWXvoxvwCfvDNuM8Nh4F"
)

if result.is_success():
  RenderJSON(json.loads(result.text))
elif result.is_error():
  print(result.errors)

<ipython-input-40-aeb7951d9269>:2: DeprecatedWarning: retrieve_transaction is deprecated
  result = client.transactions.retrieve_transaction(


In [ ]:
# refund
# ref: https://developer.squareup.com/explorer/square/refunds-api/refund-payment
# so far, the API only support refund based on amount
# i.e., the itemized refund require further develpment to wrap the chosen item as a totoal amount
# and write a note (or some other tracking method) to point to the refund items. 
# Note: Square terminal already implement itemized refund using similar wrapper idea mentioned above.

# Note: the implementation of refund in real world scenario can be much more complicated.
# e.g., a customer comes with a latte and claim it has been too much sugar and asks for refund.
# the barista might just process the request anyways without asking about proof of purchase, like receipt or
# card information. While it is suggested that to always trace back the purchase information (e.g., payment_id).
# So, the workflow is more intact.

# In this demo, assuming:
# - the occurance of return is very little that to affect the overall transactions.
# - the casher can always trace back the original payment_id (using receipt, or tender info).

# module refund:

In [56]:
from datetime import date

result = client.payments.list_payments(begin_time="2021-01-01T00:00:00.007Z",
                             end_time="2021-06-01T00:00:00.007Z")
if result.is_success():
#   RenderJSON(json.loads(result.text))
    pass
elif result.is_error():
  print(result.errors)


result = client.payments.get_payment(payment_id="7Z2gqg12ldVdymOHio9uwRKVFxNZY")
if result.is_success():
#   RenderJSON(json.loads(result.text))
    pass
elif result.is_error():
  print(result.errors)
order_id = result.body.get("payment").get("order_id")

result = client.orders.retrieve_order(order_id=order_id)
if result.is_success():
  RenderJSON(json.loads(result.text))
elif result.is_error():
  print(result.errors)

# result
# "order": ⊖{
#         "id": "wYxuBFr3Z2sq65QpNuzWBImRZc4F",
#         "location_id": "LCN0KMDE4Y5WR",
#         "line_items": ⊕[2 items],
#         "created_at": "2021-05-27T23:41:47.782Z",
#         "updated_at": "2021-05-27T23:41:50.000Z",
#         "state": "COMPLETED",
#         "version": 4,
#         "total_tax_money": ⊕{2 items},
#         "total_discount_money": ⊕{2 items},
#         "total_tip_money": ⊕{2 items},
#         "total_money": ⊕{2 items},
#         "closed_at": "2021-05-27T23:41:48.223Z",
#         "tenders": ⊕[1 item],
#         "total_service_charge_money": ⊕{2 items},
#         "net_amounts": ⊕{5 items},
#         "source": ⊕{1 item}
#     }

# continue

In [75]:
# prep items to return
# require
# - payment id (from earlier)
# - line_items info (catalog_object_id, base_price_money.amount, quantity(as max refunable))
# - quatity of line_items to return 

# line_items, (log this)
line_items = result.body.get("order").get("line_items")
quantity_to_refund_list = [1, 2]

def line_items_trim(line_items):
    return [[item.get("catalog_object_id"), 
                       item.get("base_price_money").get("amount"),
                       item.get("quantity"),
                       item.get("name"),
                       item.get("variation_name")] for item in line_items]
line_items_refunds = [item_info for item_info in zip(line_items_trim(line_items), quantity_to_refund_list)]
# note_refunds, (log this)
note_refunds = [f"{item_refunds[1]} x {item_refunds[0][4]}" for item_refunds in line_items_refunds]
amount_refunds = sum([item_refunds[0][1] * item_refunds[1] for item_refunds in line_items_refunds])



6000

In [107]:
# prep items to return V2
line_items = result.body.get("order").get("line_items")
line_items_refund = line_items

quantity_to_refund_list = [1,2]
for (item, quant) in zip(line_items_new, quantity_to_refund_list):
    item["quantity"] = quant
line_items_refund

amount_refunds = sum(
    [amount*quant for (amount,quant) in 
     zip([item.get("base_price_money").get("amount") for item in line_items], quantity_to_refund_list)])
amount_refunds

6000

In [104]:
line_items_refund = line_items
quantity_to_refund_list = [2,1]
for (item, quant) in zip(line_items_new, quantity_to_refund_list):
    item["quantity"] = quant
line_items_refund

[{'base_price_money': {'amount': 1000, 'currency': 'USD'},
  'catalog_object_id': 'IRC3NJWX5QBVIW7U736JPRCC',
  'gross_sales_money': {'amount': 4000, 'currency': 'USD'},
  'name': 'Hat',
  'quantity': 2,
  'total_discount_money': {'amount': 0, 'currency': 'USD'},
  'total_money': {'amount': 4000, 'currency': 'USD'},
  'total_tax_money': {'amount': 0, 'currency': 'USD'},
  'uid': 'RoY6zr3fO5yNWyXbZlgGK',
  'variation_name': 'Blu hat',
  'variation_total_price_money': {'amount': 4000, 'currency': 'USD'}},
 {'base_price_money': {'amount': 2500, 'currency': 'USD'},
  'catalog_object_id': 'KYZZN57QA5JHS6CWZBMWMEE6',
  'gross_sales_money': {'amount': 12500, 'currency': 'USD'},
  'name': 'Shirt',
  'quantity': 1,
  'total_discount_money': {'amount': 0, 'currency': 'USD'},
  'total_money': {'amount': 12500, 'currency': 'USD'},
  'total_tax_money': {'amount': 0, 'currency': 'USD'},
  'uid': 'pe6hyAh97ZuxDCZRSo0C0C',
  'variation_name': 'Small red shirt',
  'variation_total_price_money': {'amoun

In [76]:
# module: refunds.refund_payment
result = client.refunds.refund_payment(
  body = {
    "idempotency_key": str(uuid.uuid4()),
    "amount_money": {
      "amount": amount_refunds,
      "currency": "USD"
    },
    "payment_id": "7Z2gqg12ldVdymOHio9uwRKVFxNZY",
    "reason": "REFUND REASON"
  }
)

if result.is_success():
  RenderJSON(json.loads(result.text))
elif result.is_error():
  print(result.errors)

# result
# "refund": ⊖{
#         "id": "7Z2gqg12ldVdymOHio9uwRKVFxNZY_hiuXsma0FNoYKZMidDSsryrrCnNgcvPzZUdgR5vYhDS",
#         "status": "PENDING",
#         "amount_money": ⊖{
#             "amount": 6000,
#             "currency": "USD"
#         },
#         "payment_id": "7Z2gqg12ldVdymOHio9uwRKVFxNZY",
#         "order_id": "mJIcrOIJY5D9CD7gLk5xm24iWe4F",
#         "created_at": "2021-05-28T04:46:20.393Z",
#         "updated_at": "2021-05-28T04:46:20.393Z",
#         "location_id": "LCN0KMDE4Y5WR",
#         "reason": "REFOND REASON"
#     }

In [79]:
# before going further, it is a good time to check the status of 
# order_id(a new one) 
# payment_id(same one)

# note: Square immediately create an ad hoc order after refund COMPLETED
# however, the line_items in the ad hoc order (mJIcrOIJY5D9CD7gLk5xm24iWe4F) does not reflect itermized info
# consider using orders.update_order

result = client.payments.get_payment(payment_id="7Z2gqg12ldVdymOHio9uwRKVFxNZY")
if result.is_success():
  RenderJSON(json.loads(result.text))
#     pass
elif result.is_error():
  print(result.errors)
# order_id = result.body.get("payment").get("order_id")

# this order is the "new" order associated cooresponding to the refund
result = client.orders.retrieve_order(order_id="mJIcrOIJY5D9CD7gLk5xm24iWe4F")
if result.is_success():
  RenderJSON(json.loads(result.text))
elif result.is_error():
  print(result.errors)

# this is the original order
result = client.orders.retrieve_order(order_id="wYxuBFr3Z2sq65QpNuzWBImRZc4F")
if result.is_success():
  RenderJSON(json.loads(result.text))
elif result.is_error():
  print(result.errors)

In [123]:
# module: update ad hoc refund order, orders.update_order
# get the adhoc order to update
result_adhoc = client.orders.retrieve_order(order_id="mJIcrOIJY5D9CD7gLk5xm24iWe4F")
# RenderJSON(result_adhoc.body)
# update body part from "prep items to return V2"
result_adhoc.body["order"]["returns"][0]["return_line_items"] = line_items_refund

result = client.orders.update_order(
  order_id = "mJIcrOIJY5D9CD7gLk5xm24iWe4F",
  body = result_adhoc.body
)

if result.is_success():
  RenderJSON(result.body)
elif result.is_error():
  print(result.errors)

# [{'category': 'INVALID_REQUEST_ERROR', 'code': 'BAD_REQUEST', 
#   'detail': 'Order with id `mJIcrOIJY5D9CD7gLk5xm24iWe4F` has status `COMPLETED` and cannot be updated.'}]


[{'category': 'INVALID_REQUEST_ERROR', 'code': 'BAD_REQUEST', 'detail': 'Order with id `mJIcrOIJY5D9CD7gLk5xm24iWe4F` has status `COMPLETED` and cannot be updated.'}]


In [ ]:
# last part

In [78]:
# module: get refund info
result = client.refunds.get_payment_refund(refund_id="7Z2gqg12ldVdymOHio9uwRKVFxNZY_hiuXsma0FNoYKZMidDSsryrrCnNgcvPzZUdgR5vYhDS")
if result.is_success():
  RenderJSON(json.loads(result.text))
elif result.is_error():
  print(result.errors)